Importujemy najpierw potrzebne pakiety

In [3]:
import numpy as np
import scipy.io.wavfile
import scipy.signal
#https://dzone.com/articles/sound-synthesis-numpy
#    http://ptolemy.eecs.berkeley.edu/eecs20/week8/scale.html

In [18]:
b = np.array([["a", "b", "c"], ["s","S","s"]])
b.shape

(2, 3)

ZAPISYWANIE UTWORZONEGO SYGNALU DO PLIKU

In [35]:
scipy.io.wavfile.write('/home/rexamine/Desktop/test.wav', #plik do ktorego zapiszemy
                       fs, # the sample rate, czestotliwosc probkowania
                       np.int16(y/max(np.abs(y))*32767) #co ma byc zapisane #????? dlaczego taka normalizacja, mnozenie
                      )
#potrzebne jest przeskalowanie wektora do intow z floatow, i to sie robi ta normalizacja
data = np.random.uniform(-1,1,44100) # 44100 random samples between -1 and 1
scaled = np.int16(data/np.max(np.abs(data)) * 32767) #przeskalowany sygnal
write('test.wav', 44100, scaled) #zapisujemy go do pliku
# fs =44100 - taki zwyczaj sample rate

NameError: name 'write' is not defined

ODTWARZAMY PLIK W JUPYTERZE

In [36]:
!!aplay /home/rexamine/Desktop/test.wav

["Playing WAVE '/home/rexamine/Desktop/test.wav' : Signed 16 bit Little Endian, Rate 44100 Hz, Mono"]

TWORZENIE SYGNALU

In [34]:
f = 400 # czestotliwosc w Hz t/T
T = 1 # okres chyba 
fs = 44100 # Hz, sampling frequency, czestotliwosc probkowania
# ile bedzie probek pobranych (dyskretyzacja sygnalu)
t = np.linspace(0, T, T*fs) # czyli dzielimy okres 1 sekundy na 44100 punkty
stala = 2**(1/12) #o polton zmiana


ff = 5;   # frequency of the signal
y = sin(2*pi*ff*t)

y_A4 = np.sin(2*np.pi*f*t)
y_C5 = np.sin(2*np.pi*f*t*(stala**3))
y_E5 = np.sin(2*np.pi*f*t*(stala**7))


# DŻWIĘKI GRAJĄ PO SOBIE
y = np.concatenate((y_A4, y_C5, y_E5))
y_A4 = np.r_[np.sign(np.sin(2*np.pi*f*t)), #INNY SPOSOB
             np.sin(2*np.pi*f*t)]
# DZWIEKI GRAJA NA RAZ
y = y_A4 + y_C5 + y_E5

# ROZNE DZWIEKI
np.sign(np.sin(2*np.pi*f*t))
np.sin(2*np.pi*f*t)
scipy.signal.sawtooth(2*np.pi*f*t) # PILOKSZTALTNA?

NameError: name 'sin' is not defined

TA FUNKCJA Z ATTACK DECAY SUSTAIN

In [7]:
#attack, sustain, decey? a w procentach jako 0.1 d 0.3, t w sekundach
# T/2 - polnuta, T/4 cwiercnuta
def dzwiek(f, T, a, d):
    
    import numpy as np
    import scipy.signal
    fs = 44100 # Hz, sampling frequency
    t = np.linspace(0, T, T*fs)
    y = np.sin(2*np.pi*f*t + 0.1) #+ 0.15*scipy.signal.sawtooth(2*np.pi*f*t) ###
    y[0:int(T*fs*a)] *= np.linspace(0, 1, int(T*fs*a))
    y[-int(T*fs*d):-1] *= np.linspace(1, 0, int(T*fs*d)-1) #cos tu sie wywala
    return y

S = pow(2,1/12)
import scipy.io.wavfile
y = np.r_[dzwiek(440*S**(-2), 0.5, 0.2, 0.4),
          dzwiek(440*S**(-5), 0.5, 0.2, 0.4),
          dzwiek(440*S**(-5), 0.5, 0.2, 0.4)]
scipy.io.wavfile.write('/home/rexamine/Desktop/test.wav', #plik do ktorego zapiszemy
                       fs, # the sample rate, czestotliwosc probkowania
                       np.int16(y/max(np.abs(y))*32767) #co ma byc zapisane #????? dlaczego taka normalizacja, mnozenie
                      )
!!aplay /home/rexamine/Desktop/test.wav

["Playing WAVE '/home/rexamine/Desktop/test.wav' : Signed 16 bit Little Endian, Rate 44100 Hz, Mono"]

WCZYTYWANIE PLIKU DO JUPYTERA

In [38]:
#ZAPISUJEMY CHYBA CZESTOTLIWOSC I SAM SYGNAL
fs,y = scipy.io.wavfile.read('/home/rexamine/Desktop/padpy1516/wav_samples/Ensoniq-SQ-2-Piano-C2.wav')
print(y[0:5])
# stereo podaje lewy kanal i prawy kanal
y = np.mean(y, axis=1) # robi mono
y /= 32767 
y

[[0 0]
 [0 0]
 [0 1]
 [2 2]
 [8 7]]


array([  0.00000000e+00,   0.00000000e+00,   1.52592547e-05, ...,
         4.57777642e-05,   4.57777642e-05,   3.05185095e-05])

In [19]:
# popatrzec jak wczytuje jupyter te dzwieki i jak je zapisuje gdy robimy y =sin...

COS TAM Z LACZENIEM DZWIEKU I ROBIENIEM PAUZ COS TAM COS

In [21]:
#polaczyc to w jedno by to wpadalo dobrze jakos, przesuniecia
#trzeba zaczac od wektora pauz y= [0, 0, 0, .. ,0] i potem w odpowiednie miejsca wklejac
# pakiet librosa trzeba doinstalowac, to dodatkowe do zabawy
#rep, for ?

fs,y = scipy.io.wavfile.read('/home/rexamine/Desktop/padpy1516/wav_samples/Bass-Drum-3.wav')
fs1,y1 = scipy.io.wavfile.read('/home/rexamine/Desktop/padpy1516/wav_samples/Open-Hi-Hat-1.wav')

y = np.mean(y, axis=1) # robi mono
y /= 32767
y1 = np.mean(y1, axis=1)


def dzwiek(f, T, a, d):
    
    import numpy as np
    import scipy.signal
    fs = 44100 # Hz, sampling frequency
    t = np.linspace(0, T, T*fs)
    y = np.sin(2*np.pi*f*t + 0.1) + 0.15*scipy.signal.sawtooth(2*np.pi*f*t) ###
    y[0:int(T*fs*a)] *= np.linspace(0, 1, int(T*fs*a))
    y[-int(T*fs*a):-1] *= np.linspace(1, 0, int(T*fs*d),-1) # cos tu sie wywala
    return y

S = pow(2,1/12)
import scipy.io.wavfile
y = np.r_[dzwiek(440*S**(-2), 0.5, 0.2, 0.2),
          dzwiek(440*S**(-5), 0.5, 0.2, 0.2)]
scipy.io.wavfile.write('/home/samba/baranowskae/Desktop/test.wav', 
                           fs,
                           np.int16(y/max(np.abs(y))*32767)
                          )     
!!aplay /home/samba/baranowskae/Desktop/test.wav



/home/rexamine/anaconda/lib/python3.4/site-packages/scipy/io/wavfile.py:172: WavFileWarning: Chunk (non-data) not understood, skipping it.
  WavFileWarning)


ValueError: operands could not be broadcast together with shapes (4409,) (4410,) (4409,) 

czyli 120 bmp -> 120 cwiercnut na minute czyli 120/60 = 2, 2 cwiercnuty na sekunde (2 stukniecia)
1 takt to 4 cwiercnuty (takt to 4 stukniecia, raz dwa trzy cztery, raz dwa ...) , czyli tutaj 1 takt bedzie gral 2 sekundy, albo inaczej 1 sekunda - 2 stukniecia
60/120 - 0.5 - tyle trwa 1 cwiercnuta w tempie 120
czyli 0.5s x 4 cwiercnuty = 2 sek - tyle trwa jeden takt

Każdy takt reprezentowany jest przez 4 wiersze - czyli 1 wiersz to 1 cwiercnuta
Takt to 4 ćwierćnuty
A ćwierćnuta trwa 60/x sekund, gdzie tempo to x bpm
Czyli jak masz 120 bpm, to ćwierćnuta trwa pół sekundy, czyli takt trwa dwie
I Twój program ma puszczać 4 ścieżki naraz

wiec wnioskuje ze kolumny to kanaly a wiersze cwiercnuty
01 02 03 04
-- -- 03 --
-- 02 03 --
-- -- 03 --


### czytanie sampli tj. pliku track01.txt
01 02 03 04
-- -- 03 --
-- 02 03 --
-- -- 03 --
01 02 03 04
-- -- 03 --
-- 02 03 --
-- -- 03 --
01 02 03 04
-- -- 03 --
-- 02 03 04
-- -- 03 --
01 02 03 04
-- -- 03 04
-- 02 03 04
-- -- 03 04
gdzie 01 - odnosi się do pliku sample01.wav znajdujacego sie w tym samym katalogu co track01.txt


In [22]:
#////////////////////////////////////////////////////////////
# stare 
def wczytywane_trackow(nazwa_tracka, lokalizacja):
    #dac asserty i doctringa ''''''
    #okreslic jakos lokalizacje skad ma brac te pliki ze z tego samego folderu gdzie jest program z katalogu 
    # odpowiedniego dla utworu
    #nazwa_tracka = 01 02 jako tekst '01' '02'
    
    # robimy z '01' napis 'track01.txt' by taki plik wczytac
    plik = ''.join(['track',zlo,'.txt.'])
    track = open(plik)
    
    pierwszy_wiersz = track.readline() #jako string
    
    liczba_sampli = (len(pierwszy_wiersz) + 1) / 3 # ile na raz sampli mam puszczac
    
    
    track.close()
    
    
    #sprawdzic czy ten open i readline() dziala w ogole
    
    track = open('track01.txt')

In [19]:
# trzeba policzyc lacznie ile plik ma sekund ()
# dzwieki

zlo = '01 02 03 04 05 06 07'
len(zlo)

#y = np.mean(y, axis=1) # robi mono - zrobic MONO

# kolejne linijki (trwające 60/BMP = 60/ 120 = 0.5 sekundy) będziemy odtwarzac po kolei robiac konkatenacje
# zeby jedna linijke odtworzyc naraz zrobimy + czyli zrobie y = 01 + 02 + 03 i one wszystkie beda mialy 1 sek
# trwania? (albo zamienie je tak by trwaly sekunde to jest jak mam jedno uderze) teraz pytanie jak
# strescic te sekunde grania do 0,5 sekundy?

# tak wybiore pierwsze 0.5 sekundy dzwieku z = y[0:0.5*44100] (T - czas razy probkowanie 44100)


# jak sprawic by cos co trwa 1 sek trwalo 0,25 sek

20

In [2]:
def wczytywanie_piosenki(nazwa_piosenki = "song.txt"):
    # 
    #dac asserty i doctringa ''''''
    #okreslic jakos lokalizacje skad ma brac te pliki ze z tego samego folderu gdzie jest program z katalogu 
    # odpowiedniego dla utworu - jesli w ogole, bo jedno to wczytanie pliku o takiej nazwie, a drugie skad - 
    # chyba automatycznie wczytuje z miejsca skad uruchomil sie python??
    """
    bedzie wczytywac zawartosc pliku song.txt do np.array zlozonego z macierzy n (liczba wierszy w song.txt) 
    na 1 - 1 wiersz bedzie odpowiadal jednemu trackowi - zapisane to bedzie jako string
    
    arg:
    nazwa_piosenki - nazwa pliku tekstowego wraz z rozszerzeniem, typ: str, assert - ze nie podano stringa
    reszta (ze podano zla sciezke zajmie sie genfromtxt)
    """
    
    import numpy as np

    
    song = np.genfromtxt(nazwa_piosenki,dtype='str')
    
    
    return song

In [3]:
a = wczytywanie_piosenki()
# czy nie bedzie trzeba wczytywac tego z folder /utwor1/track01.txt !!!!!!!!!!

In [11]:
def wczytywanie_sciezek(nazwa_piosenki):
    #dac asserty i doctringa ''''''
    #okreslic jakos lokalizacje skad ma brac te pliki ze z tego samego folderu gdzie jest program z katalogu 
    # odpowiedniego dla utworu
    
    # robimy z '01' napis 'track01.txt' by taki plik wczytac
    """
    bedziemy na podstawie pliku song.txt wczytywac tracki, typ: numpy.ndarray, i zapiszemy je jako jedna 
    macierz - zlaczymy calosc
    
    
    
    assert, ze nie ten typ
    
    arg:
    nazwa_piosenki - np. array
    
    wyjscie:
    macierz typu np.ndarray kazdy wiersz to jedna cwiercnuta, elementy wiersza jako stringi
    """
    import numpy as np
    
    
    
    # ta funkcja zwraca np.ndarray, ktorego elementy sa juz posortowane; zakladam, ze sciezki ktore, sa podane
    # w song.txt maja numery odpowiadajace ich liczbie, tj. jak mam 3 rozne sciezki w song.txt to ich nazwy to
    # track01.txt, track02.txt, track03.txt, a nie np. track11.txt, track01.txt, track07.txt, to zalozenie
    # potrzebne dalej
    jakie_sciezki = np.unique(nazwa_piosenki)
    
    ile_sciezek = len(jakie_sciezki)
    
    #lista zwykla do trzymania sciezek, sciezka[0] - track01 (iterator trza cofnac)
    sciezki = []
    
    
    for ktora_sciezka in jakie_sciezki: # w iteratorze bierzemy napis "01" "02" itd. string!!!
        plik = ''.join(['track',ktora_sciezka,'.txt']) # tworzymy napis z nazwa pliku tracka,tj "track01.txt"
        #iterator ktora_sciezka przebiega po kolei numery (posortowane sa po unique)
        s = np.genfromtxt(plik, dtype='str')
        sciezki.append(s) # wczytujemy dane z niego
    
          
    piosenka = []
    for ktora_sciezka in nazwa_piosenki:       
        piosenka.append(sciezki[int(ktora_sciezka)-1])
    
    #zliczamy ile jest kanalow ( kolumn w trackach)
    ile_kanalow = sciezki[0].shape[1] # licze ile kolumn w plikach jest
    
    #zliczamy ile mamy cwiercnut
    n = 0
    for i in nazwa_piosenki:
        
        n += sciezki[int(i)-1].shape[0]
    
    #tworzymy z listy typ np.ndarray, ktory bedzie mial odpowiednie wymiary
    piosenka = np.array(piosenka).reshape(n, ile_kanalow)
    
    
    
    
    
    return piosenka
    
    
# machnąc w forze ze jak ktorys z trackow ma rozna liczbe kolumn to wywal blad albo
# sprawdzanie co jest w piluk to juz inna funkcja


1

In [12]:
wczytywanie_sciezek(a)

array([['01', '02', '03', '04'],
       ['--', '--', '03', '--'],
       ['--', '02', '03', '--'],
       ['--', '--', '03', '--'],
       ['01', '02', '03', '04'],
       ['--', '--', '03', '--'],
       ['--', '02', '03', '--'],
       ['--', '--', '03', '--'],
       ['01', '02', '03', '04'],
       ['--', '--', '03', '--'],
       ['--', '02', '03', '04'],
       ['--', '--', '03', '--'],
       ['01', '02', '03', '04'],
       ['--', '--', '03', '04'],
       ['--', '02', '03', '04'],
       ['--', '--', '03', '04'],
       ['01', '02', '03', '04'],
       ['01', '--', '03', '--'],
       ['--', '02', '03', '--'],
       ['--', '--', '03', '--'],
       ['01', '02', '03', '04'],
       ['01', '--', '03', '--'],
       ['--', '02', '03', '--'],
       ['--', '--', '03', '--'],
       ['01', '02', '03', '04'],
       ['--', '--', '03', '--'],
       ['01', '02', '03', '04'],
       ['--', '--', '03', '--'],
       ['01', '02', '03', '04'],
       ['--', '--', '03', '04'],
       ['-

In [80]:
sciezka = np.genfromtxt("track01.txt", dtype='str')
lista = []
lista.append(sciezka) 
lista.append(sciezka) 
np.array(lista)
lista[0].reshape(32,2)
sciezka
# jak rozdzielic te wiersze 

array([['01', '02', '03', '04'],
       ['--', '--', '03', '--'],
       ['--', '02', '03', '--'],
       ['--', '--', '03', '--'],
       ['01', '02', '03', '04'],
       ['--', '--', '03', '--'],
       ['--', '02', '03', '--'],
       ['--', '--', '03', '--'],
       ['01', '02', '03', '04'],
       ['--', '--', '03', '--'],
       ['--', '02', '03', '04'],
       ['--', '--', '03', '--'],
       ['01', '02', '03', '04'],
       ['--', '--', '03', '04'],
       ['--', '02', '03', '04'],
       ['--', '--', '03', '04']], 
      dtype='<U2')